# Tópicos de Industria I
## Práctica 3 - Métricas de Caminados aleatorios

<img style="margin: 10px" src="./public/banner.png" alt="Assignment Banner" height="180" width="980" />

**E-mail:** roberto.carrillo7958@alumnos.udg.mx

**Ciclo:** 2023-A

**Fecha:** 17/02/2023

## MODULES

In [2]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

### Implementación

 1. Se tienen que cargar las trayectorias, donde usamos un archivo csv para transformarlo a Dataframe
 2. Se define una función que permite calcular la distancia entre dos puntos:
      1. Calculamos el cuadrado de la diferencia del eje correspondiente
      2. Retornamos la raíz de la suma de dichos cuadrados
 3. Creamos una función que itera sobre cada posición de la trayectoria y calcula la distancia de cada punto, retornando la suma acumulativa de toda la trayectoria
 4. Se genera una función para graficar, y se renderiza

In [4]:
# Load existing trajectories
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [5]:
# Creación de función para cálculo de distancia entre dos vectores
def calc_dist_2vectors(point1, point2):

    x_diff = np.square(point2.x_pos - point1.x_pos)
    y_diff = np.square(point2.y_pos - point1.y_pos)
    vectors_distance = np.sqrt(x_diff + y_diff)
    
    return vectors_distance

# Creación de función para calcular la longitud de la trayectoria
# Haciendo uso de la función creada anteriormente, vamos iterando sobre cada posición
# Y acumulando la suma de cada distancia entre el vector actual y el anterior
def create_pl_bm_2d(trajectory_df):
    pl_distance_bm_df = np.array([
        calc_dist_2vectors(
            trajectory_df.iloc[i - 1],
            trajectory_df.iloc[i]
        ) for i in range(1, trajectory_df.shape[0])
    ])
    return np.cumsum(pl_distance_bm_df)


In [14]:
# Define trajectories array
trajectories = [
    { "traj_name": "BM_2d_df_3", "traj_data": BM_2d_df_3 },
    { "traj_name": "BM_2d_df_6", "traj_data": BM_2d_df_6 },
    { "traj_name": "CRW_2d_df_9", "traj_data": CRW_2d_df_9 }
]
trajectories_df = pd.DataFrame(columns=['traj_name', 'traj_data', 'path_length', 'path_length_data'])

for traj in trajectories:
    path_length = create_pl_bm_2d(traj['traj_data'])
    temp_df = pd.DataFrame([{
        "traj_name": traj['traj_name'],
        "traj_data": traj['traj_data'],
        "path_length": path_length[-1],
        "path_length_data": path_length,
    }])
    trajectories_df = pd.concat([trajectories_df, temp_df], axis=0, ignore_index=True)



In [25]:
# Plotting
# Init figure
fig_path_length = go.Figure()

def plot_path_length(path_length_data, traj_name, fig):
    fig.add_trace(go.Scatter(
        x = np.arange(len(path_length_data)),
        y = path_length_data,
        marker = dict( size = 2 ),
        line = dict( width = 2 ),
        name = f"{traj_name}",
        showlegend = True
    ))

for index, row in trajectories_df.iterrows():
    plot_path_length(row.path_length_data, row.traj_name, fig_path_length)

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (8 pts)

* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**


El Mean Squared Displacement es una métrica que entrega más información acerca de la trayectoria de una partícula. Mientras que el largo del recorrido se encarga de medir toda la distancia que se recorrió a lo largo de la trayectoria, el MSD es el valor promedio qu

### Implementación

    1. Se genera una función que calcule el


In [15]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
        y = BM_2d_df_6.y_pos,
        z = BM_2d_df_6.index,
        marker = dict(size=2),
        line = dict(color='blue', width=2),
        mode = 'lines',
        name = 'BM 2d',
        showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
        y = CRW_2d_df_9.y_pos,
        z = CRW_2d_df_9.index,
        marker = dict(size=2),
        line = dict(color='red', width=2),
        mode = 'lines',
        name = 'CRW 2d',
        showlegend = True))

fig_3d.show()

In [39]:
def calc_displacement_vec(trajectory, n_step):
    
    return np.array([ 
        np.square(distance.euclidean(trajectory.iloc[i - n_step], trajectory.iloc[i])) 
            for i in range(n_step, trajectory.shape[0], 1)
    ])

def create_msd_vec(trajectory, n_step):
    for tau in range(1, trajectory.shape[0]):
        n = tau
        msd = np.mean()

for tau in range(1,BM_2d_df_6.shape[0]):
    # BM speed = 6
    n = tau
    msd = np.mean(np.array([np.square(np.sqrt(np.square((BM_2d_df_6.x_pos[j] - BM_2d_df_6.x_pos[j-n])) + np.square((BM_2d_df_6.y_pos[j] - BM_2d_df_6.y_pos[j-n])))) for j in range(n,len(BM_2d_df_6))]))
    MSD_BM = np.append(MSD_BM,msd)

array([1.10173181e+01, 9.73571744e+01, 1.26272521e+02, 2.26195939e+01,
       2.35043983e+01, 2.29988131e+00, 1.15529566e+02, 3.21978840e+01,
       1.43891172e+02, 8.34299758e-02, 1.07221891e+02, 7.76613920e+01,
       9.55043785e+01, 7.52471372e+01, 6.64017468e+01, 1.22716665e+02,
       5.73261141e+01, 1.42584154e+02, 1.37283036e+02, 1.02363653e+02,
       1.10496493e+02, 3.42880603e+01, 1.17171631e+02, 2.53124652e+01,
       4.31801792e-01, 3.00881116e-01, 1.12260456e+02, 1.21201282e+02,
       2.30962872e+01, 5.44038013e+01, 1.43999909e+02, 1.46133577e+01,
       5.43559607e+01, 1.37978725e+00, 1.03122272e+02, 1.40822616e+02,
       1.43746230e+02, 1.45204842e+00, 9.31618310e+01, 1.37853462e+00,
       7.79420052e+01, 9.51373631e+01, 3.82891350e+01, 1.42666147e+02,
       1.43721356e+02, 1.28053140e+02, 1.13889808e+01, 2.05396045e+01,
       8.51108740e+01, 8.15872363e+00, 1.09418806e+02, 1.19831605e+02,
       2.97679906e+01, 1.24449607e+02, 1.18988492e+02, 1.39533426e+02,
      

In [ ]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here
    
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
    
## end - Add your code here


fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (9 pts)

* Generar CRWs con dos exponentes diferentes
* Guardar trayectorias en Pandas Data Frame
* Obtener Turning-angle distribution
* Guardar metricas en **Pandas** Data Frame
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [56]:
# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [83]:
def calc_2vec_angle(p1, p2, p3):
    # Calc ab, bc and it's norm values
    ab = p2 - p1
    bc = p3 - p2
    norm_ab = np.linalg.norm(ab)
    norm_bc = np.linalg.norm(bc)

    # Calc dot and cross product of both segments
    dot_p = np.dot(ab, bc)
    cross_p = np.cross(ab, bc)
    # Check direction of vector
    direction = np.sign(cross_p)
    if direction == 0:
        direction = 1
    # Get cos angle, add eps to avoid NaN    
    cos_angle = dot_p/((norm_ab * norm_bc) + np.finfo(float).eps)
    turn_angle = np.arccos(cos_angle) * direction
    return turn_angle

def calc_turn_angle_dist(trajectory):
    # Generate out array
    out_array = np.empty(0)
    # Iterate over trajectory, get turn angle and append to out_array
    for index in range(1, trajectory.shape[0] - 1):
        ## start - Add your code here
        p1 = trajectory.iloc[index-1]
        p2 = trajectory.iloc[index]
        p3 = trajectory.iloc[index+1]
        calculated_angle = calc_2vec_angle(p1,p2,p3)
        out_array= np.append(out_array, calculated_angle)
    return out_array

# Create aux arrays to store angle values
aux_ta_CRW_2d_df_6 = calc_turn_angle_dist(CRW_2d_df_6)
aux_ta_CRW_2d_df_9 = calc_turn_angle_dist(CRW_2d_df_9)

# Save to pandas DF
# start - Add your code here
met_crw_df= pd.DataFrame()
met_crw_df['TA_CRW_6']=aux_ta_CRW_2d_df_6
temp_crw_df= pd.DataFrame()
temp_crw_df['TA_CRW_9']=aux_ta_CRW_2d_df_9
met_crw_6_9_df=pd.merge(met_crw_df,temp_crw_df, left_index=True, right_index=True,how='inner')
## end - Add your code here


# # Write to csv
# ## start - Add your code here
# met_crw_6_9_df.to_csv("./met_crw_6_9.csv")
## end - Add your code here

In [84]:
def calc_2vec_angle(p1, p2, p3):
    # Calc ab, bc and it's norm values
    ab = p2 - p1
    bc = p3 - p2
    norm_ab = np.linalg.norm(ab)
    norm_bc = np.linalg.norm(bc)

    # Calc dot and cross product of both segments
    dot_p = np.dot(ab, bc)
    cross_p = np.cross(ab, bc)
    # Check direction of vector
    direction = np.sign(cross_p)
    if direction == 0:
        direction = 1
    # Get cos angle, add eps to avoid NaN    
    cos_angle = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    turn_angle = np.arccos(np.around(cos_angle,4)) * direction
    return turn_angle

def calc_turn_angle_dist(trajectory):
    # Generate out array
    out_array = np.empty(0)
    # Iterate over trajectory, get turn angle and append to out_array
    for index, _row in CRW_2d_df_6[1:-1].iterrows():
        ## start - Add your code here
        p1 = trajectory.iloc[index-1]
        p2 = trajectory.iloc[index]
        p3 = trajectory.iloc[index+1]
        calculated_angle = calc_2vec_angle(p1,p2,p3)
        out_array= np.append(out_array, calculated_angle)
    return out_array


aux_ta_CRW_2d_df_6 = calc_turn_angle_dist(CRW_2d_df_6)
aux_ta_CRW_2d_df_9 = calc_turn_angle_dist(CRW_2d_df_9)

    
# Save to pandas DF
## start - Add your code here
met_crw_df= pd.DataFrame()
met_crw_df['TA_CRW_6']=aux_ta_CRW_2d_df_6
temp_crw_df= pd.DataFrame()
temp_crw_df['TA_CRW_9']=aux_ta_CRW_2d_df_9
met_crw_6_9_df=pd.merge(met_crw_df,temp_crw_df, left_index=True, right_index=True,how='inner')
## end - Add your code here


# Write to csv
## start - Add your code here
met_crw_6_9_df.to_csv("./met_crw_6_9.csv")
## end - Add your code here

In [85]:
# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(x=met_crw_6_9_df.TA_CRW_6,
                                    xbins=dict(size=np.pi/300),
                                    histnorm='percent',
                                    name='observed_0.6',
                                    marker_color='#EB89B5',
                                    opacity=0.80,
                                    showlegend=True))
## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_df_3.add_trace(go.Histogram(x=met_crw_6_9_df.TA_CRW_9,
                                    xbins=dict(size=np.pi/300),
                                    histnorm='percent',
                                    name='observed_0.9',
                                    marker_color='#880E4F',
                                    opacity=0.30,
                                    showlegend=True))

## end - Add your code here


# Add origin distributions
## start - Add your code here
resolution=300

aux_domain = np.linspace(0, 2*np.pi, resolution)

for CRW_exponent in [0.6, 0.9]:
  wrapcauchy_pdf = np.array([wrapcauchy.pdf(i, CRW_exponent) for i in aux_domain])

  #another aux vector
  aux_plot = np.linspace(-np.pi, np.pi, resolution)

  #Only for plotting
  plot_wrapcauchy_pdf = np.concatenate((wrapcauchy_pdf[int (resolution/2):resolution], wrapcauchy_pdf[0:int(resolution/2)]),  axis=0)
  fig_met_df_3.add_trace(go.Scatter(x = aux_plot, 
                                   y = plot_wrapcauchy_pdf,
                                   marker = dict(size=2),
                                   line = dict(width=2),
                                   mode = 'lines',
                                   name = 'cauchy_{}'.format(CRW_exponent),
                                   showlegend = True))
## end - Add your code here
fig_met_df_3.update_layout(
    title_text='Turning-angle Distribution - (Dist. origen vs Dist. observada)'
)
fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (9 pts)

* Generar Levy Flight
* Guardar trayectorias en **Pandas** Data Frame
* Guardar metricas en **numpy** array
* Obtener Step-length distribution
* Comparar en gráfica distribución origen vs distribución observada
* Visualizar con **plotly**

In [ ]:
# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# aux to store turning angles
aux_ta_Levy_2d_df_1 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))

## start - Add your code here
# Sumar cada paso en una sola dirección,
# producto punto por la multiplicación de eps
# Sign function numpy
    
## end - Add your code here


# aux to store turning angles
aux_ta_Levy_2d_df_7 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))

## start - Add your code here
    
## end - Add your code here

In [ ]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
    
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
    
## end - Add your code here


# Add origin distributions
## start - Add your code here
    
## end - Add your code here


fig_met_df_4.show()